In [2]:
import pandas as pd
import requests, json
import time

In [3]:
police = pd.read_csv("./data_preprocess/police_구_동_좌표추가.csv")
cctv = pd.read_csv("./data_preprocess/cctv_동네추가.csv")
light = pd.read_csv("./data_preprocess/light_동_추가.csv")
safe = pd.read_csv("./data_preprocess/safe.csv")

In [3]:
safe = safe.drop(["Unnamed: 0"], axis=1)
police = police.drop(["Unnamed: 0"], axis=1)
cctv = cctv.drop(["Unnamed: 0"], axis=1)
light = light.drop(["Unnamed: 0"], axis=1)

In [4]:
police

,연번,시도청,경찰서,관서명,구분,전화번호,주소
0,1,서울청,서울중부,을지,지구대,02-2279-1908,서울특별시 중구 을지로 234
1,2,서울청,서울중부,광희,지구대,02-2233-1444,서울특별시 중구 퇴계로 375-1
2,3,서울청,서울중부,약수,지구대,02-2234-8112,"서울특별시 중구 동호로 5길 15, 약수지구대"
3,4,서울청,서울중부,신당,파출소,02-2252-0435,"서울특별시 중구 다산로 248 (신당동, 신당파출소)"
4,5,서울청,서울중부,장충,파출소,02-2274-9003,서울특별시 중구 동호로 261
...,...,...,...,...,...,...,...
238,239,서울청,서울수서,대치,지구대,02-538-1312,서울특별시 강남구 삼성로 203
239,240,서울청,서울수서,수서,파출소,02-445-0112,서울특별시 강남구 밤고개로1길 48
240,241,서울청,서울수서,대왕,파출소,02-3411-0323,서울특별시 강남구 헌릉로 623
241,242,서울청,서울수서,일원,지구대,02-3411-0112,서울특별시 강남구 일원로3길 35 일원파출소


In [4]:
cctv.head(3)

,번호,자치구명,안심주소,CCTV용도,위도,경도,CCTV수량,수정일시,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명
0,2,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434221,126.903997,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청
1,3,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434303,126.903990,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청
2,4,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434312,126.904080,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청


In [6]:
light.head(3)

,번호,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명
0,10212,강일동068,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동137-3,37.576149,127.169137,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청
1,10213,강일동069,1,서울특별시 강동구 아리수로93가길 223,서울특별시 강동구 강일동139,37.576176,127.168800,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청
2,10214,강일동070,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동26-3,37.576840,127.168722,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청


In [7]:
safe.head(3)

,기준년도,시도,시군구,행정동,경찰서명,인구,합계원표,성범죄원표,살인강도원표,교통범죄원표,...,기타내용확인불가112,기타화재112,기타구조요청112,기타소음112,기타노점상112,기타기타타기관112,기타서비스요청112,기타청탁금지법112,기타재해재난112,기타기타위험동물112
0,2018,서울특별시,용산구,용문동,서울용산,54808.0,10017,331,16,2284,...,3637,447,882,3253,68,2172,424,0,0,0
1,2018,서울특별시,성동구,왕십리제2동,서울성동,17748.0,8129,126,14,1831,...,2897,354,647,1914,22,1750,256,0,0,0
2,2018,서울특별시,성동구,금호1가동,서울성동,16087.0,8129,126,14,1831,...,2897,354,647,1914,22,1750,256,0,0,0


# police(경찰관서 위치 현황) 데이터 전처리

### 정규표현식 -> 경찰서 주소에서 () 부분 제거, 단순 주소만 나오게 전처리

In [96]:
import re

sample_add = "서울특별시 종로구 송월길 167(교북동 10-38)"
cleaned_address = re.sub(r'\([^)]*\)', '', sample_add)

print(cleaned_address)

서울특별시 종로구 송월길 167


In [100]:
# 정규표현식을 활용하여 address 열에서 괄호와 괄호 이하의 내용 제거
police['주소'] = police['주소'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))

In [101]:
police.iloc[:20]

,연번,시도청,경찰서,관서명,구분,전화번호,주소
0,1,서울청,서울중부,을지,지구대,02-2279-1908,서울특별시 중구 을지로 234
1,2,서울청,서울중부,광희,지구대,02-2233-1444,서울특별시 중구 퇴계로 375-1
2,3,서울청,서울중부,약수,지구대,02-2234-8112,"서울특별시 중구 동호로 5길 15, 약수지구대"
3,4,서울청,서울중부,신당,파출소,02-2252-0435,서울특별시 중구 다산로 248
4,5,서울청,서울중부,장충,파출소,02-2274-9003,서울특별시 중구 동호로 261
5,6,서울청,서울중부,충무,파출소,02-2278-7710,서울특별시 중구 퇴계로 178
6,7,서울청,서울중부,을지로3가,파출소,02-2266-2404,서울특별시 중구 충무로56-1
7,8,서울청,서울종로,종로2가,지구대,02-3701-4301,서울특별시 종로구 종로17길 4 종로2가파출소
8,9,서울청,서울종로,관수,파출소,02-3701-4302,서울특별시 종로구 삼일대로 386
9,10,서울청,서울종로,세검정,파출소,02-3701-4507,서울특별시 종로구 세검정로 226


In [95]:
def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    headers = {"Authorization": "KakaoAK f49ecb903ce3ea3655d37548b914df7c"}
    api_json = json.loads(str(requests.get(url,headers=headers).text))
    return api_json

In [81]:
police["주소"][0]

'서울특별시 중구  을지로 234'

In [107]:
sample = get_location(police["주소"][0])

In [110]:
sample["documents"][0]["address"]["x"]

'127.003943654119'

In [13]:
address = [police["주소"][x] for x in range(len(police["주소"]))]
len(address)

243

In [97]:
sample = [police["주소"][x] for x in range(10)]
# sample[1]

In [98]:
sample_list = []
for i in range(len(sample)):
    ad_json = get_location(sample[i])
    sample_list.append(ad_json["documents"][0]["address"]["region_2depth_name"])

In [99]:
sample_list

['중구', '중구', '중구', '중구', '중구', '중구', '중구', '종로구', '종로구', '종로구']

In [31]:
address = [police["주소"][x] for x in range(len(police["주소"]))]
address[14:17]

['서울특별시 종로구  자하문로17길 24', '서울특별시 종로구  송월길 167', '서울특별시 중구  세종대로 141']

In [88]:
from tqdm.notebook import tqdm

address_list = []
# error_list = ["행촌동", "장안동", "신길동", "장위동", "풍납동", "도봉동"]
for i in tqdm(range(len(address))):
    try:
        ad_json = get_location(address[i])
        address_list.append(ad_json["documents"][0]["address"]["region_3depth_name"])
    except IndexError:
        print(f"{i+1}번째에서 IndexError 발생! 주소: {address[i]}")
        continue

  0%|          | 0/243 [00:00<?, ?it/s]

16번째에서 IndexError 발생! 주소: 서울특별시 종로구  송월길 167
63번째에서 IndexError 발생! 주소: 서울특별시 동대문구 한천로20길 7
79번째에서 IndexError 발생! 주소: 서울특별시 영등포구 신길동 4533-16


KeyboardInterrupt: 

In [86]:
len(address_list)

237

In [45]:
address[62]

'서울특별시 동대문구 한천로20길 7'

- 16번째에서 IndexError 발생! 주소: 서울특별시 종로구  송월길 167 -> 행촌동
- 63번째에서 IndexError 발생! 주소: 서울특별시 동대문구 한천로20길 7 -> 장안동
- 79번째에서 IndexError 발생! 주소: 서울특별시 영등포구 신길동 4533-16 -> 신길동
- 177번째에서 IndexError 발생! 주소: 서울특별시 성북구 장위로174 장위지구대 -> 장위동
- 210번째에서 IndexError 발생! 주소: 서울특별시 송파구  토성로 64 -> 풍납동
- 234번째에서 IndexError 발생! 주소: 서울특별시 도봉구  도봉로169길 43 도봉1파출소 -> 도봉동

IndexError 발생 -> error_list에서 하나씩 꺼내와서 address_list에 채워줌

In [89]:
from tqdm.notebook import tqdm

address_list = []
error_list = ["행촌동", "장안동", "신길동", "장위동", "풍납동", "도봉동"]
error_index = 0  # error_list의 인덱스를 추적하기 위한 변수

for i in tqdm(range(len(address))):
    try:
        ad_json = get_location(address[i])
        address_list.append(ad_json["documents"][0]["address"]["region_3depth_name"])
    except IndexError:
        print(f"{i+1}번째에서 IndexError 발생! 주소: {address[i]}")
        if error_index < len(error_list):
            address_list.append(error_list[error_index])
            error_index += 1
        else:
            address_list.append(None)  # error_list가 모두 사용된 경우 None을 추가하여 처리
            print("None 추가")
        continue

  0%|          | 0/243 [00:00<?, ?it/s]

16번째에서 IndexError 발생! 주소: 서울특별시 종로구  송월길 167
63번째에서 IndexError 발생! 주소: 서울특별시 동대문구 한천로20길 7
79번째에서 IndexError 발생! 주소: 서울특별시 영등포구 신길동 4533-16
177번째에서 IndexError 발생! 주소: 서울특별시 성북구 장위로174 장위지구대
210번째에서 IndexError 발생! 주소: 서울특별시 송파구  토성로 64
234번째에서 IndexError 발생! 주소: 서울특별시 도봉구  도봉로169길 43 도봉1파출소


In [90]:
print(address[233])
print(address_list[233])

서울특별시 도봉구  도봉로169길 43 도봉1파출소
도봉동


police 데이터에 '동네' 열 추가 -> address_list

In [91]:
police["동네"] = address_list

In [92]:
police.to_csv("./data_preprocess/police.csv")

In [102]:
def get_gu_name(row, gu_error_list):
    try:
        location = get_location(row["주소"])
        return location["documents"][0]["address"]["region_2depth_name"]
    except IndexError:
        gu_error_list.append(row["주소"])
        return None

tqdm.pandas()
gu_error_list = []
police["gu"] = police.progress_apply(lambda row: get_gu_name(row, gu_error_list), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████| 243/243 [00:36<00:00,  6.71it/s]


In [104]:
def get_dong_name(row, dong_error_list):
    try:
        location = get_location(row["주소"])
        return location["documents"][0]["address"]["region_3depth_name"]
    except IndexError:
        dong_error_list.append(row["주소"])
        return None

tqdm.pandas()
dong_error_list = []
police["dong"] = police.progress_apply(lambda row: get_dong_name(row, dong_error_list), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████| 243/243 [00:30<00:00,  7.88it/s]


In [105]:
police

,연번,시도청,경찰서,관서명,구분,전화번호,주소,gu,dong
0,1,서울청,서울중부,을지,지구대,02-2279-1908,서울특별시 중구 을지로 234,중구,을지로5가
1,2,서울청,서울중부,광희,지구대,02-2233-1444,서울특별시 중구 퇴계로 375-1,중구,신당동
2,3,서울청,서울중부,약수,지구대,02-2234-8112,"서울특별시 중구 동호로 5길 15, 약수지구대",중구,신당동
3,4,서울청,서울중부,신당,파출소,02-2252-0435,서울특별시 중구 다산로 248,중구,신당동
4,5,서울청,서울중부,장충,파출소,02-2274-9003,서울특별시 중구 동호로 261,중구,장충동2가
...,...,...,...,...,...,...,...,...,...
238,239,서울청,서울수서,대치,지구대,02-538-1312,서울특별시 강남구 삼성로 203,강남구,대치동
239,240,서울청,서울수서,수서,파출소,02-445-0112,서울특별시 강남구 밤고개로1길 48,강남구,수서동
240,241,서울청,서울수서,대왕,파출소,02-3411-0323,서울특별시 강남구 헌릉로 623,강남구,세곡동
241,242,서울청,서울수서,일원,지구대,02-3411-0112,서울특별시 강남구 일원로3길 35 일원파출소,강남구,일원동


In [106]:
police.to_csv("./data_preprocess/police_구_동_추가.csv")

## x 좌표 추가

In [112]:
police = pd.read_csv("./data_preprocess/police_구_동_추가.csv")
police

,Unnamed: 0,연번,시도청,경찰서,관서명,구분,전화번호,주소,gu,dong
0,0,1,서울청,서울중부,을지,지구대,02-2279-1908,서울특별시 중구 을지로 234,중구,을지로5가
1,1,2,서울청,서울중부,광희,지구대,02-2233-1444,서울특별시 중구 퇴계로 375-1,중구,신당동
2,2,3,서울청,서울중부,약수,지구대,02-2234-8112,"서울특별시 중구 동호로 5길 15, 약수지구대",중구,신당동
3,3,4,서울청,서울중부,신당,파출소,02-2252-0435,서울특별시 중구 다산로 248,중구,신당동
4,4,5,서울청,서울중부,장충,파출소,02-2274-9003,서울특별시 중구 동호로 261,중구,장충동2가
...,...,...,...,...,...,...,...,...,...,...
238,238,239,서울청,서울수서,대치,지구대,02-538-1312,서울특별시 강남구 삼성로 203,강남구,대치동
239,239,240,서울청,서울수서,수서,파출소,02-445-0112,서울특별시 강남구 밤고개로1길 48,강남구,수서동
240,240,241,서울청,서울수서,대왕,파출소,02-3411-0323,서울특별시 강남구 헌릉로 623,강남구,세곡동
241,241,242,서울청,서울수서,일원,지구대,02-3411-0112,서울특별시 강남구 일원로3길 35 일원파출소,강남구,일원동


In [113]:
def get_x_point(row, x_error_list):
    try:
        location = get_location(row["주소"])
        return location["documents"][0]["address"]["x"]
    except IndexError:
        x_error_list.append(row["주소"])
        return None

tqdm.pandas()
x_error_list = []
police["x"] = police.progress_apply(lambda row: get_x_point(row, x_error_list), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████| 243/243 [00:35<00:00,  6.86it/s]


In [114]:
police

,Unnamed: 0,연번,시도청,경찰서,관서명,구분,전화번호,주소,gu,dong,x
0,0,1,서울청,서울중부,을지,지구대,02-2279-1908,서울특별시 중구 을지로 234,중구,을지로5가,127.003943654119
1,1,2,서울청,서울중부,광희,지구대,02-2233-1444,서울특별시 중구 퇴계로 375-1,중구,신당동,127.013376356837
2,2,3,서울청,서울중부,약수,지구대,02-2234-8112,"서울특별시 중구 동호로 5길 15, 약수지구대",중구,신당동,127.0123820902
3,3,4,서울청,서울중부,신당,파출소,02-2252-0435,서울특별시 중구 다산로 248,중구,신당동,127.01638496365
4,4,5,서울청,서울중부,장충,파출소,02-2274-9003,서울특별시 중구 동호로 261,중구,장충동2가,127.004802045236
...,...,...,...,...,...,...,...,...,...,...,...
238,238,239,서울청,서울수서,대치,지구대,02-538-1312,서울특별시 강남구 삼성로 203,강남구,대치동,127.062617624125
239,239,240,서울청,서울수서,수서,파출소,02-445-0112,서울특별시 강남구 밤고개로1길 48,강남구,수서동,127.104813214036
240,240,241,서울청,서울수서,대왕,파출소,02-3411-0323,서울특별시 강남구 헌릉로 623,강남구,세곡동,127.10736243987
241,241,242,서울청,서울수서,일원,지구대,02-3411-0112,서울특별시 강남구 일원로3길 35 일원파출소,강남구,일원동,127.083373556556


## y 좌표 추가

In [115]:
def get_y_point(row, y_error_list):
    try:
        location = get_location(row["주소"])
        return location["documents"][0]["address"]["y"]
    except IndexError:
        y_error_list.append(row["주소"])
        return None

tqdm.pandas()
y_error_list = []
police["y"] = police.progress_apply(lambda row: get_y_point(row, y_error_list), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████| 243/243 [00:31<00:00,  7.75it/s]


In [116]:
police

,Unnamed: 0,연번,시도청,경찰서,관서명,구분,전화번호,주소,gu,dong,x,y
0,0,1,서울청,서울중부,을지,지구대,02-2279-1908,서울특별시 중구 을지로 234,중구,을지로5가,127.003943654119,37.5664734170529
1,1,2,서울청,서울중부,광희,지구대,02-2233-1444,서울특별시 중구 퇴계로 375-1,중구,신당동,127.013376356837,37.5652684538307
2,2,3,서울청,서울중부,약수,지구대,02-2234-8112,"서울특별시 중구 동호로 5길 15, 약수지구대",중구,신당동,127.0123820902,37.5521442775352
3,3,4,서울청,서울중부,신당,파출소,02-2252-0435,서울특별시 중구 다산로 248,중구,신당동,127.01638496365,37.5650442671787
4,4,5,서울청,서울중부,장충,파출소,02-2274-9003,서울특별시 중구 동호로 261,중구,장충동2가,127.004802045236,37.5587633776035
...,...,...,...,...,...,...,...,...,...,...,...,...
238,238,239,서울청,서울수서,대치,지구대,02-538-1312,서울특별시 강남구 삼성로 203,강남구,대치동,127.062617624125,37.494873946034
239,239,240,서울청,서울수서,수서,파출소,02-445-0112,서울특별시 강남구 밤고개로1길 48,강남구,수서동,127.104813214036,37.489127606674
240,240,241,서울청,서울수서,대왕,파출소,02-3411-0323,서울특별시 강남구 헌릉로 623,강남구,세곡동,127.10736243987,37.4651760424395
241,241,242,서울청,서울수서,일원,지구대,02-3411-0112,서울특별시 강남구 일원로3길 35 일원파출소,강남구,일원동,127.083373556556,37.4924161491256


## x, y좌표 추가한 csv 파일 저장

In [117]:
police.to_csv("./data_preprocess/police_구_동_좌표추가.csv")

# cctv 데이터(서울시 CCTV 데이터) 전처리

In [9]:
cctv

,번호,자치구명,안심주소,CCTV용도,위도,경도,CCTV수량,수정일시,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명
0,2,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434221,126.903997,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청
1,3,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434303,126.903990,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청
2,4,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434312,126.904080,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청
3,5,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434383,126.903963,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청
4,6,서초구,내곡동 (신원동) 36-1,방범,37.434617,127.069873,1,2021-01-19,서울서초경찰서,서울서초,G30000001329813,서울청
...,...,...,...,...,...,...,...,...,...,...,...,...
61513,61515,도봉구,도봉동 8번지 다락원체육공원,방범,37.692401,127.046621,2,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청
61514,61516,도봉구,창포원로 45(도봉2동 8다락원 체육공원),방범,37.692401,127.046621,1,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청
61515,61517,도봉구,창포원로 45(도봉2동 8다락원 체육공원),방범,37.692464,127.047108,1,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청
61516,61518,도봉구,창포원로 45(도봉2동 8다락원 체육공원),방범,37.692900,127.047000,1,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청


In [5]:
X = list(cctv["경도"])
Y = list(cctv["위도"])

# api 키 변경

In [15]:
# 좌표 데이터로 주소 변환하기
# api_key: 905ba1855aa03b3416e81d27e5331850
def get_location(X, Y):
    url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={X}&y={Y}"
    headers = {"Authorization": "KakaoAK f49ecb903ce3ea3655d37548b914df7c"}
    api_json = json.loads(str(requests.get(url,headers=headers).text))
    return api_json

In [7]:
sample_x, sample_y = cctv.iloc[0]["경도"], cctv.iloc[0]["위도"]
print(sample_x, sample_y)

126.9039971 37.4342208


In [8]:
cctv.iloc[0]

번호                                   2
자치구명                               금천구
안심주소       시흥대로2길 8-49 주변 [시흥3동 978-6]
CCTV용도                              방범
위도                           37.434221
경도                          126.903997
CCTV수량                               1
수정일시                        2022-02-23
경찰서명                           서울금천경찰서
경찰서부서명                            서울금천
경찰서부서코드                G30000001332675
시도경찰청명                             서울청
Name: 0, dtype: object

In [14]:
sample_result = get_location(127.084823, 37.590240)
sample_result = sample_result
sample_result

{'errorType': 'RequestThrottled', 'message': 'API limit has been exceeded.'}

In [83]:
X = [cctv["경도"][x] for x in range(len(cctv["경도"]))]
Y = [cctv["위도"][x] for x in range(len(cctv["위도"]))]
print(len(X), len(Y))

61518 61518


In [48]:
# 일단 보류!

# cctv_add_list = []
# cctv_error_list = []
# for i in tqdm(range(len(cctv))):
#     try:
#         ad_json = get_location(X[i], Y[i])
#         cctv_add_list.append(ad_json["documents"][0]["address"]["region_3depth_name"])
#     except IndexError:
#         print(f"{i}번째에서 IndexError 발생! 위치: {X[i], Y[i]}")
#         cctv_error_list.append([X[i], Y[i]])
#         print(cctv_error_list)
#         continue

## Sample 데이터셋 돌리기

In [49]:
# sample_set["동네"] = None

In [12]:
# sample 데이터셋 뽑기
sample_set = cctv.sample(n=10, random_state=1234)
sample_set

,번호,자치구명,안심주소,CCTV용도,위도,경도,CCTV수량,수정일시,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명
47334,47336,중랑구,면목2동 138-60(보조),방범,37.590240,127.084823,1,2022-04-28,서울중랑경찰서,서울중랑,G30000001329641,서울청
5739,5741,금천구,남부순환로108길 27 [가산동145-15],방범,37.477712,126.892015,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청
4762,4764,관악구,DD011 1-4R 남현동 602-22,방범,37.475382,126.980513,1,2021-09-08,서울관악경찰서,서울관악,G30000001329692,서울청
8819,8821,서초구,서초3동 1459-9,방범,37.483860,127.012859,1,2021-01-19,서울서초경찰서,서울서초,G30000001329813,서울청
29192,29194,강서구,C301048_화곡8동 159-7(고정2) - 후암아트빌 방향,방범,37.531424,126.851333,1,2021-01-29,서울강서경찰서,서울강서,G30000001329716,서울청
14350,14352,강남구,[일원1-218] (2) 일원로5길90(643)_2,방범,37.492902,127.086637,1,2022-06-23,서울수서경찰서,서울수서,G30000001329960,서울청
18016,18018,동작구,상도4동 251-8 (주),방범,37.499673,126.933764,1,2021-01-20,서울동작경찰서,서울동작,G30000001332634,서울청
9850,9852,구로구,1494.개봉3동 298-75 핀홀,도심공원,37.485529,126.847366,1,2022-02-17,서울구로경찰서,서울구로,G30000001329787,서울청
60465,60467,강북구,우이동 21-16,방범,37.661217,127.012700,1,2022-04-18,서울강북경찰서,서울강북,G30000001332662,서울청
31267,31269,성동구,C663_(고정4)6_이마트,도심공원,37.540195,127.049568,1,2021-12-07,서울성동경찰서,서울성동,G30000001329479,서울청


In [13]:
sample_X = [sample_set.iloc[x]["경도"] for x in range(len(sample_set))]
sample_Y = [sample_set.iloc[x]["위도"] for x in range(len(sample_set))]

In [14]:
sample_X[-1]

127.0495682

In [17]:
sample_set["동네"] = None

In [27]:
from tqdm import tqdm


sample_error_list = []

# for i in tqdm(range(len(sample_set))):
#     try:
#         ad_json = get_location(sample_X[i], sample_Y[i])
#         # print(ad_json["documents"][0]["address"]["region_3depth_name"])
#         sample_set["동네"] = ad_json["documents"][0]["address"]["region_3depth_name"]
#     except IndexError:
#         print(f"{i}번째에서 IndexError 발생! 위치: {sample_X[i], sample_Y[i]}")
#         sample_error_list.append([sample_X[i], sample_Y[i]])
#         sample_set
#         continue

# tqdm.pandas()
# sample_set["행정동"] = sample_set[["경도", "위도"]].progress_apply(lambda x, y: get_location(x, y))
tqdm.pandas()
sample_set["행정동"] = sample_set.progress_apply(lambda row: get_location(row["경도"], row["위도"]), axis=1)


100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  7.92it/s]


In [39]:
sample_set.iloc[0]["행정동"]["documents"][0]["address"]["region_1depth_name"]
sample_set["si"] = 

'서울'

In [46]:
tqdm.pandas()
def get_si_name(row):
    location = get_location(row["경도"], row["위도"])
    return location["documents"][0]["address"]["region_1depth_name"]

sample_set["si"] = sample_set.progress_apply(get_si_name, axis=1)

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  7.76it/s]


In [47]:
tqdm.pandas()

def get_gu_name(row):
    location = get_location(row["경도"], row["위도"])
    return location["documents"][0]["address"]["region_2depth_name"]

sample_set["gu"] = sample_set.progress_apply(get_gu_name, axis=1)

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  7.55it/s]


In [49]:
tqdm.pandas()

def get_dong_name(row):
    location = get_location(row["경도"], row["위도"])
    return location["documents"][0]["address"]["region_3depth_name"]

sample_set["dong"] = sample_set.progress_apply(get_dong_name, axis=1)

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  8.26it/s]


In [50]:
sample_set

,번호,자치구명,안심주소,CCTV용도,위도,경도,CCTV수량,수정일시,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,동네,행정동,si,gu,dong
47334,47336,중랑구,면목2동 138-60(보조),방범,37.590240,127.084823,1,2022-04-28,서울중랑경찰서,서울중랑,G30000001329641,서울청,성수동1가,"{'meta': {'total_count': 1}, 'documents': [{'r...",서울,중랑구,면목동
5739,5741,금천구,남부순환로108길 27 [가산동145-15],방범,37.477712,126.892015,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,성수동1가,"{'meta': {'total_count': 1}, 'documents': [{'r...",서울,금천구,가산동
4762,4764,관악구,DD011 1-4R 남현동 602-22,방범,37.475382,126.980513,1,2021-09-08,서울관악경찰서,서울관악,G30000001329692,서울청,성수동1가,"{'meta': {'total_count': 1}, 'documents': [{'r...",서울,관악구,남현동
8819,8821,서초구,서초3동 1459-9,방범,37.483860,127.012859,1,2021-01-19,서울서초경찰서,서울서초,G30000001329813,서울청,성수동1가,"{'meta': {'total_count': 1}, 'documents': [{'r...",서울,서초구,서초동
29192,29194,강서구,C301048_화곡8동 159-7(고정2) - 후암아트빌 방향,방범,37.531424,126.851333,1,2021-01-29,서울강서경찰서,서울강서,G30000001329716,서울청,성수동1가,"{'meta': {'total_count': 1}, 'documents': [{'r...",서울,강서구,화곡동
14350,14352,강남구,[일원1-218] (2) 일원로5길90(643)_2,방범,37.492902,127.086637,1,2022-06-23,서울수서경찰서,서울수서,G30000001329960,서울청,성수동1가,"{'meta': {'total_count': 1}, 'documents': [{'r...",서울,강남구,일원동
18016,18018,동작구,상도4동 251-8 (주),방범,37.499673,126.933764,1,2021-01-20,서울동작경찰서,서울동작,G30000001332634,서울청,성수동1가,"{'meta': {'total_count': 1}, 'documents': [{'r...",서울,동작구,상도동
9850,9852,구로구,1494.개봉3동 298-75 핀홀,도심공원,37.485529,126.847366,1,2022-02-17,서울구로경찰서,서울구로,G30000001329787,서울청,성수동1가,"{'meta': {'total_count': 1}, 'documents': [{'r...",서울,구로구,개봉동
60465,60467,강북구,우이동 21-16,방범,37.661217,127.012700,1,2022-04-18,서울강북경찰서,서울강북,G30000001332662,서울청,성수동1가,"{'meta': {'total_count': 1}, 'documents': [{'r...",서울,강북구,우이동
31267,31269,성동구,C663_(고정4)6_이마트,도심공원,37.540195,127.049568,1,2021-12-07,서울성동경찰서,서울성동,G30000001329479,서울청,성수동1가,"{'meta': {'total_count': 1}, 'documents': [{'r...",서울,성동구,성수동1가


## 전체 데이터셋 돌리기

In [34]:
cctv.iloc[0]["위도"]

37.4342208

In [35]:
X = [cctv.iloc[x]["경도"] for x in range(len(cctv))]
Y = [cctv.iloc[x]["위도"] for x in range(len(cctv))]

In [38]:
Y

[37.4342208,
 37.43430303,
 37.43431204,
 37.43438346,
 37.4346168,
 37.4346177,
 37.43461925,
 37.43462204,
 37.43462687,
 37.43468727,
 37.43468896,
 37.43471371,
 37.43472174,
 37.43474594,
 37.43475094,
 37.43475313,
 37.43475575,
 37.43476857,
 37.4347817,
 37.43481979,
 37.43484529,
 37.43489116,
 37.43493678,
 37.43501759,
 37.43505231,
 37.43509186,
 37.43509694,
 37.43512135,
 37.43538335,
 37.43547953,
 37.43560011,
 37.43560124,
 37.43560347,
 37.43561131,
 37.4361116,
 37.43611363,
 37.43612453,
 37.43615563,
 37.43624534,
 37.43731186,
 37.43747984,
 37.43787091,
 37.43818716,
 37.43818716,
 37.43819621,
 37.43822321,
 37.43822556,
 37.43832761,
 37.43885237,
 37.4390208,
 37.43902094,
 37.43905293,
 37.43926797,
 37.43930348,
 37.43934445,
 37.43936509,
 37.43941499,
 37.43942389,
 37.43965216,
 37.43967805,
 37.43992377,
 37.43992707,
 37.43993077,
 37.43993789,
 37.43995394,
 37.43996915,
 37.4399701,
 37.44006643,
 37.44009496,
 37.44016114,
 37.44026435,
 37.44032568,

## 시 컬럼 추가

In [59]:
def get_si_name(row, error_list):
    try:
        location = get_location(row["경도"], row["위도"])
        return location["documents"][0]["address"]["region_1depth_name"]
    except IndexError:
        error_list.append([row["경도"], row["위도"]])
        return None

tqdm.pandas()
error_list = []
cctv["si"] = cctv.progress_apply(lambda row: get_si_name(row, error_list), axis=1)

100%|███████████████████████████████████████████████████████████████████████████| 61518/61518 [2:28:34<00:00,  6.90it/s]


In [60]:
cctv.isnull().sum()

번호          0
자치구명        0
안심주소        0
CCTV용도     75
위도          0
경도          0
CCTV수량      0
수정일시        0
경찰서명        0
경찰서부서명      0
경찰서부서코드     0
시도경찰청명      0
si         79
dtype: int64

## 구 컬럼 추가

In [68]:
cctv["gu"] = None

In [62]:
def get_gu_name(row, gu_error_list):
    try:
        location = get_location(row["경도"], row["위도"])
        return location["documents"][0]["address"]["region_2depth_name"]
    except IndexError:
        gu_error_list.append([row["경도"], row["위도"]])
        return None

tqdm.pandas()
gu_error_list = []
cctv["gu"] = cctv.progress_apply(lambda row: get_gu_name(row, gu_error_list), axis=1)

 19%|██████████████▎                                                            | 11722/61518 [29:23<2:04:52,  6.65it/s]


KeyboardInterrupt: 

## 동 컬럼 추가 (여기 돌려 동 추가 해!)

In [11]:
from tqdm import tqdm

In [17]:
def get_dong_name(row, dong_error_list):
    try: 
        location = get_location(row["경도"], row["위도"])
        return location["documents"][0]["address"]["region_3depth_name"]
    except IndexError:
        dong_error_list.append([row["경도"], row["위도"]])
        return None

tqdm.pandas()
dong_error_list = []
cctv["dong"] = cctv.progress_apply(lambda row: get_dong_name(row, dong_error_list), axis=1)

100%|███████████████████████████████████████████████████████████████████████████| 61518/61518 [2:08:51<00:00,  7.96it/s]


한 번에 될 줄 알았지만 사고다... 일단 아래 코드는 보류

In [18]:
cctv

,번호,자치구명,안심주소,CCTV용도,위도,경도,CCTV수량,수정일시,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong
0,2,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434221,126.903997,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동
1,3,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434303,126.903990,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동
2,4,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434312,126.904080,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동
3,5,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434383,126.903963,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동
4,6,서초구,내곡동 (신원동) 36-1,방범,37.434617,127.069873,1,2021-01-19,서울서초경찰서,서울서초,G30000001329813,서울청,신원동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61513,61515,도봉구,도봉동 8번지 다락원체육공원,방범,37.692401,127.046621,2,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동
61514,61516,도봉구,창포원로 45(도봉2동 8다락원 체육공원),방범,37.692401,127.046621,1,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동
61515,61517,도봉구,창포원로 45(도봉2동 8다락원 체육공원),방범,37.692464,127.047108,1,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동
61516,61518,도봉구,창포원로 45(도봉2동 8다락원 체육공원),방범,37.692900,127.047000,1,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동


In [21]:
cctv.isnull().sum()

번호          0
자치구명        0
안심주소        0
CCTV용도     75
위도          0
경도          0
CCTV수량      0
수정일시        0
경찰서명        0
경찰서부서명      0
경찰서부서코드     0
시도경찰청명      0
dong       79
dtype: int64

# 위 코드 다 돌아갔으면 일단 csv 파일로 저장부터 해!
### 일단 저장하고 null 값 처리하자~

In [19]:
cctv.to_csv("./cctv_동네추가.csv")

# 체감안전 데이터 수정

In [24]:
safe = safe.iloc[:, :15]

In [26]:
safe.to_csv("./data_preprocess/safe.csv")

# 보안등 데이터 행정동 추가

In [4]:
light.head(3)

,Unnamed: 0.1,Unnamed: 0,번호,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong
0,0,0,10212,강일동068,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동137-3,37.576149,127.169137,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동
1,1,1,10213,강일동069,1,서울특별시 강동구 아리수로93가길 223,서울특별시 강동구 강일동139,37.576176,127.168800,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동
2,2,2,10214,강일동070,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동26-3,37.576840,127.168722,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동


In [54]:
def get_location(X, Y):
    url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={X}&y={Y}"
    headers = {"Authorization": "KakaoAK 905ba1855aa03b3416e81d27e5331850"}
    api_json = json.loads(str(requests.get(url,headers=headers).text))
    return api_json

## 전체를 한 번에 돌리고 싶은데 하루 사용량이 10만건 밖에 되질 않음
#### 그래서 일단 6000건만 돌려보려고 함.

In [18]:
# from tqdm import tqdm

# def get_dong_name(row, dong_error_list):
#     try: 
#         location = get_location(row["경도"], row["위도"])
#         return location["documents"][0]["address"]["region_3depth_name"]
#     except IndexError:
#         dong_error_list.append([row["경도"], row["위도"]])
#         return None

# tqdm.pandas()
# dong_error_list = []
# light["dong"] = light.progress_apply(lambda row: get_dong_name(row, dong_error_list), axis=1)

### 여기로 올라와

In [8]:
# 전체 데이터프레임을 처리할 범위 지정
start_index = 170000

# subset_df 생성
subset_df = light.iloc[start_index:]
subset_df

,Unnamed: 0.1,Unnamed: 0,번호,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong
170000,170000,170000,89545,08-662,1,서울특별시 동작구 남부순환로259가길 15-8,서울특별시 동작구 사당동1050-20,37.476818,126.973000,2020.0,건축물,02-820-1408,서울특별시 동작구청,2022-07-07,서울동작경찰서,서울동작,G30000001332634,서울청,NaN
170001,170001,170001,89546,09-3001,1,서울특별시 동작구 동작대로29길 110,서울특별시 동작구 사당동105,37.489841,126.977994,2011.0,전용주,02-820-1408,서울특별시 동작구청,2022-07-07,서울동작경찰서,서울동작,G30000001332634,서울청,NaN
170002,170002,170002,89547,09-3002,1,서울특별시 동작구 동작대로29길 81,서울특별시 동작구 사당동105,37.488831,126.977451,2011.0,전용주,02-820-1408,서울특별시 동작구청,2022-07-07,서울동작경찰서,서울동작,G30000001332634,서울청,NaN
170003,170003,170003,89548,09-3003,1,서울특별시 동작구 동작대로29길 81,서울특별시 동작구 사당동105,37.488831,126.977451,2011.0,전용주,02-820-1408,서울특별시 동작구청,2022-07-07,서울동작경찰서,서울동작,G30000001332634,서울청,NaN
170004,170004,170004,89549,09-3004,1,서울특별시 동작구 동작대로29길 81,서울특별시 동작구 사당동105,37.488831,126.977451,2011.0,전용주,02-820-1408,서울특별시 동작구청,2022-07-07,서울동작경찰서,서울동작,G30000001332634,서울청,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195696,195696,195696,155194,영등포본동-96,1,서울특별시 영등포구 영신로20길 14,서울특별시 영등포구 영등포동 618-157,37.515420,126.909194,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,NaN
195697,195697,195697,155195,영등포본동-97,1,서울특별시 영등포구 신길로61길,NaN,37.498164,126.907974,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,NaN
195698,195698,195698,155196,영등포본동-98,1,서울특별시 영등포구 신길로61길 17,서울특별시 영등포구 영등포동 592-70,37.514667,126.909340,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,NaN
195699,195699,195699,155197,영등포본동-99,1,서울특별시 영등포구 영신로20가길 13,서울특별시 영등포구 영등포동 592-56,37.514771,126.909017,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,NaN


In [9]:
from tqdm import tqdm

def get_dong_name(row, sub_dong_error):
    try: 
        location = get_location(row["경도"], row["위도"])
        return location["documents"][0]["address"]["region_3depth_name"]
    except IndexError:
        sub_dong_error.append([row["경도"], row["위도"]])
        return None

tqdm.pandas()
sub_dong_error = []
subset_df["dong"] = subset_df.progress_apply(lambda row: get_dong_name(row, sub_dong_error), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████| 25701/25701 [51:35<00:00,  8.30it/s]
/var/folders/bc/bsqk8v9x20jgf1jsh_fbh7z40000gn/T/ipykernel_14595/422736972.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["dong"] = subset_df.progress_apply(lambda row: get_dong_name(row, sub_dong_error), axis=1)


In [10]:
subset_df

,Unnamed: 0.1,Unnamed: 0,번호,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong
170000,170000,170000,89545,08-662,1,서울특별시 동작구 남부순환로259가길 15-8,서울특별시 동작구 사당동1050-20,37.476818,126.973000,2020.0,건축물,02-820-1408,서울특별시 동작구청,2022-07-07,서울동작경찰서,서울동작,G30000001332634,서울청,사당동
170001,170001,170001,89546,09-3001,1,서울특별시 동작구 동작대로29길 110,서울특별시 동작구 사당동105,37.489841,126.977994,2011.0,전용주,02-820-1408,서울특별시 동작구청,2022-07-07,서울동작경찰서,서울동작,G30000001332634,서울청,사당동
170002,170002,170002,89547,09-3002,1,서울특별시 동작구 동작대로29길 81,서울특별시 동작구 사당동105,37.488831,126.977451,2011.0,전용주,02-820-1408,서울특별시 동작구청,2022-07-07,서울동작경찰서,서울동작,G30000001332634,서울청,사당동
170003,170003,170003,89548,09-3003,1,서울특별시 동작구 동작대로29길 81,서울특별시 동작구 사당동105,37.488831,126.977451,2011.0,전용주,02-820-1408,서울특별시 동작구청,2022-07-07,서울동작경찰서,서울동작,G30000001332634,서울청,사당동
170004,170004,170004,89549,09-3004,1,서울특별시 동작구 동작대로29길 81,서울특별시 동작구 사당동105,37.488831,126.977451,2011.0,전용주,02-820-1408,서울특별시 동작구청,2022-07-07,서울동작경찰서,서울동작,G30000001332634,서울청,사당동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195696,195696,195696,155194,영등포본동-96,1,서울특별시 영등포구 영신로20길 14,서울특별시 영등포구 영등포동 618-157,37.515420,126.909194,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,영등포동
195697,195697,195697,155195,영등포본동-97,1,서울특별시 영등포구 신길로61길,NaN,37.498164,126.907974,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,신길동
195698,195698,195698,155196,영등포본동-98,1,서울특별시 영등포구 신길로61길 17,서울특별시 영등포구 영등포동 592-70,37.514667,126.909340,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,영등포동
195699,195699,195699,155197,영등포본동-99,1,서울특별시 영등포구 영신로20가길 13,서울특별시 영등포구 영등포동 592-56,37.514771,126.909017,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,영등포동


In [11]:
add_light = light[:]
add_light

,Unnamed: 0.1,Unnamed: 0,번호,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong
0,0,0,10212,강일동068,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동137-3,37.576149,127.169137,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동
1,1,1,10213,강일동069,1,서울특별시 강동구 아리수로93가길 223,서울특별시 강동구 강일동139,37.576176,127.168800,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동
2,2,2,10214,강일동070,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동26-3,37.576840,127.168722,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동
3,3,3,10215,강일동071,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동20,37.576904,127.168868,NaN,전용주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동
4,4,4,10216,강일동072,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동20,37.576968,127.168992,NaN,전용주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195696,195696,195696,155194,영등포본동-96,1,서울특별시 영등포구 영신로20길 14,서울특별시 영등포구 영등포동 618-157,37.515420,126.909194,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,NaN
195697,195697,195697,155195,영등포본동-97,1,서울특별시 영등포구 신길로61길,NaN,37.498164,126.907974,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,NaN
195698,195698,195698,155196,영등포본동-98,1,서울특별시 영등포구 신길로61길 17,서울특별시 영등포구 영등포동 592-70,37.514667,126.909340,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,NaN
195699,195699,195699,155197,영등포본동-99,1,서울특별시 영등포구 영신로20가길 13,서울특별시 영등포구 영등포동 592-56,37.514771,126.909017,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,NaN


In [12]:
dong_list = subset_df["dong"].tolist()
dong_list[-20:]

['영등포동',
 '영등포동',
 '영등포동',
 '영등포동',
 '영등포동',
 '영등포동',
 '영등포동',
 '영등포동',
 '영등포동',
 '영등포동',
 '영등포동',
 '영등포동',
 '영등포동',
 '영등포동',
 '영등포동',
 '영등포동',
 '신길동',
 '영등포동',
 '영등포동',
 '신길동']

### add_light에 dong 추가

In [15]:
# light 데이터프레임의 start_index부터 end_index 전까지의 범위에 대해 "dong" 열에 dong_list 추가
add_light.iloc[start_index:, add_light.columns.get_loc("dong")] = dong_list

In [16]:
add_light.iloc[start_index-2:]

,Unnamed: 0.1,Unnamed: 0,번호,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong
169998,169998,169998,89543,08-487,1,서울특별시 동작구 남부순환로265길 3,서울특별시 동작구 사당동1048-39,37.476813,126.975255,2011.0,전용주,02-820-1408,서울특별시 동작구청,2022-07-07,서울동작경찰서,서울동작,G30000001332634,서울청,사당동
169999,169999,169999,89544,08-660,1,서울특별시 동작구 남부순환로261길 20,서울특별시 동작구 사당동1048-15,37.477475,126.974047,2011.0,전용주,02-820-1408,서울특별시 동작구청,2022-07-07,서울동작경찰서,서울동작,G30000001332634,서울청,사당동
170000,170000,170000,89545,08-662,1,서울특별시 동작구 남부순환로259가길 15-8,서울특별시 동작구 사당동1050-20,37.476818,126.973000,2020.0,건축물,02-820-1408,서울특별시 동작구청,2022-07-07,서울동작경찰서,서울동작,G30000001332634,서울청,사당동
170001,170001,170001,89546,09-3001,1,서울특별시 동작구 동작대로29길 110,서울특별시 동작구 사당동105,37.489841,126.977994,2011.0,전용주,02-820-1408,서울특별시 동작구청,2022-07-07,서울동작경찰서,서울동작,G30000001332634,서울청,사당동
170002,170002,170002,89547,09-3002,1,서울특별시 동작구 동작대로29길 81,서울특별시 동작구 사당동105,37.488831,126.977451,2011.0,전용주,02-820-1408,서울특별시 동작구청,2022-07-07,서울동작경찰서,서울동작,G30000001332634,서울청,사당동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195696,195696,195696,155194,영등포본동-96,1,서울특별시 영등포구 영신로20길 14,서울특별시 영등포구 영등포동 618-157,37.515420,126.909194,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,영등포동
195697,195697,195697,155195,영등포본동-97,1,서울특별시 영등포구 신길로61길,NaN,37.498164,126.907974,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,신길동
195698,195698,195698,155196,영등포본동-98,1,서울특별시 영등포구 신길로61길 17,서울특별시 영등포구 영등포동 592-70,37.514667,126.909340,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,영등포동
195699,195699,195699,155197,영등포본동-99,1,서울특별시 영등포구 영신로20가길 13,서울특별시 영등포구 영등포동 592-56,37.514771,126.909017,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,영등포동


### 전체 데이터(light)에 추가

In [18]:
# 전체 데이터프레임과 subset_df의 "dong" 열 합치기
light.iloc[start_index:, light.columns.get_loc("dong")] = dong_list
light

,Unnamed: 0.1,Unnamed: 0,번호,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong
0,0,0,10212,강일동068,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동137-3,37.576149,127.169137,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동
1,1,1,10213,강일동069,1,서울특별시 강동구 아리수로93가길 223,서울특별시 강동구 강일동139,37.576176,127.168800,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동
2,2,2,10214,강일동070,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동26-3,37.576840,127.168722,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동
3,3,3,10215,강일동071,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동20,37.576904,127.168868,NaN,전용주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동
4,4,4,10216,강일동072,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동20,37.576968,127.168992,NaN,전용주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195696,195696,195696,155194,영등포본동-96,1,서울특별시 영등포구 영신로20길 14,서울특별시 영등포구 영등포동 618-157,37.515420,126.909194,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,영등포동
195697,195697,195697,155195,영등포본동-97,1,서울특별시 영등포구 신길로61길,NaN,37.498164,126.907974,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,신길동
195698,195698,195698,155196,영등포본동-98,1,서울특별시 영등포구 신길로61길 17,서울특별시 영등포구 영등포동 592-70,37.514667,126.909340,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,영등포동
195699,195699,195699,155197,영등포본동-99,1,서울특별시 영등포구 영신로20가길 13,서울특별시 영등포구 영등포동 592-56,37.514771,126.909017,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,영등포동


In [19]:
light.to_csv("./light_주소추가.csv")

In [22]:
light = pd.read_csv("./light_주소추가.csv")

In [24]:
light = light.drop(["Unnamed: 0.2", "Unnamed: 0.1", "Unnamed: 0"], axis=1)

In [27]:
light.to_csv("./data_preprocess/light_동_추가.csv")

# cctv 데이터 gu 컬럼 추가

In [10]:
cctv = pd.read_csv("./data_preprocess/cctv_동네추가.csv")
cctv

,Unnamed: 0,번호,자치구명,안심주소,CCTV용도,위도,경도,CCTV수량,수정일시,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong
0,0,2,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434221,126.903997,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동
1,1,3,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434303,126.903990,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동
2,2,4,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434312,126.904080,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동
3,3,5,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434383,126.903963,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동
4,4,6,서초구,내곡동 (신원동) 36-1,방범,37.434617,127.069873,1,2021-01-19,서울서초경찰서,서울서초,G30000001329813,서울청,신원동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61513,61513,61515,도봉구,도봉동 8번지 다락원체육공원,방범,37.692401,127.046621,2,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동
61514,61514,61516,도봉구,창포원로 45(도봉2동 8다락원 체육공원),방범,37.692401,127.046621,1,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동
61515,61515,61517,도봉구,창포원로 45(도봉2동 8다락원 체육공원),방범,37.692464,127.047108,1,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동
61516,61516,61518,도봉구,창포원로 45(도봉2동 8다락원 체육공원),방범,37.692900,127.047000,1,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동


In [11]:
cctv = cctv.drop(["Unnamed: 0"], axis=1)
cctv

,번호,자치구명,안심주소,CCTV용도,위도,경도,CCTV수량,수정일시,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong
0,2,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434221,126.903997,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동
1,3,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434303,126.903990,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동
2,4,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434312,126.904080,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동
3,5,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434383,126.903963,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동
4,6,서초구,내곡동 (신원동) 36-1,방범,37.434617,127.069873,1,2021-01-19,서울서초경찰서,서울서초,G30000001329813,서울청,신원동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61513,61515,도봉구,도봉동 8번지 다락원체육공원,방범,37.692401,127.046621,2,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동
61514,61516,도봉구,창포원로 45(도봉2동 8다락원 체육공원),방범,37.692401,127.046621,1,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동
61515,61517,도봉구,창포원로 45(도봉2동 8다락원 체육공원),방범,37.692464,127.047108,1,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동
61516,61518,도봉구,창포원로 45(도봉2동 8다락원 체육공원),방범,37.692900,127.047000,1,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동


In [12]:
cctv["gu"] = cctv["자치구명"]

In [14]:
cctv = cctv.drop(["자치구명"], axis=1)
cctv

,번호,안심주소,CCTV용도,위도,경도,CCTV수량,수정일시,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong,gu
0,2,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434221,126.903997,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동,금천구
1,3,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434303,126.903990,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동,금천구
2,4,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434312,126.904080,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동,금천구
3,5,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434383,126.903963,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동,금천구
4,6,내곡동 (신원동) 36-1,방범,37.434617,127.069873,1,2021-01-19,서울서초경찰서,서울서초,G30000001329813,서울청,신원동,서초구
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61513,61515,도봉동 8번지 다락원체육공원,방범,37.692401,127.046621,2,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동,도봉구
61514,61516,창포원로 45(도봉2동 8다락원 체육공원),방범,37.692401,127.046621,1,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동,도봉구
61515,61517,창포원로 45(도봉2동 8다락원 체육공원),방범,37.692464,127.047108,1,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동,도봉구
61516,61518,창포원로 45(도봉2동 8다락원 체육공원),방범,37.692900,127.047000,1,2022-05-17,서울도봉경찰서,서울도봉,G30000001329939,서울청,도봉동,도봉구


In [15]:
cctv["si"] = "서울시"

In [17]:
cctv.head(3)

,번호,안심주소,CCTV용도,위도,경도,CCTV수량,수정일시,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong,gu,si
0,2,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434221,126.903997,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동,금천구,서울시
1,3,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434303,126.903990,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동,금천구,서울시
2,4,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434312,126.904080,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동,금천구,서울시


In [18]:
cctv.to_csv("./data_preprocess/cctv_수정.csv", index=False)

# light gu 컬럼 추가

In [20]:
light.head(3)

,Unnamed: 0,번호,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong
0,0,10212,강일동068,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동137-3,37.576149,127.169137,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동
1,1,10213,강일동069,1,서울특별시 강동구 아리수로93가길 223,서울특별시 강동구 강일동139,37.576176,127.168800,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동
2,2,10214,강일동070,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동26-3,37.576840,127.168722,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동


In [22]:
import re

sample = "서울특별시 관악구 청룡11길 36-16"
gu_pattern = r'(\S+구)'  # "관악구"를 추출하기 위한 정규표현식 패턴

match = re.search(gu_pattern, sample)
if match:
    gu = match.group(1)
    print(gu)  # 결과: "관악구"

관악구


In [24]:
light_sample = light.sample(n=10, random_state=1234)
light_sample

,Unnamed: 0,번호,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong
77699,77699,52914,자양4동223,1,서울특별시 광진구 뚝섬로22가길 58-5,서울특별시 광진구 자양동 57-65,37.534648,127.064156,NaN,한전주,02-450-7873,서울특별시 광진구청,2022-07-01,서울광진경찰서,서울광진,G30000001332648,서울청,자양동
190439,190439,149936,도림동-597,1,서울특별시 영등포구 도신로15다길 3,서울특별시 영등포구 도림동 232-12,37.507050,126.898234,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,도림동
17758,17758,79506,2009-10-03,1,서울특별시 도봉구 창제1동 661-24,서울특별시 도봉구 창제1동 661-24,37.649755,127.035970,2008.0,한전주,02-2091-4083,도봉구청,2018-05-01,서울도봉경찰서,서울도봉,G30000001329939,서울청,창동
137261,137261,103554,홍제1동,1,서울특별시 서대문구 통일로39길 80-1,서울특별시 서대문구 홍제1동312-94,37.586962,126.940749,NaN,건축물,02-330-1275,서울특별시 서대문구청,2020-01-01,서울서대문경찰서,서울서대문,G30000001329309,서울청,홍제동
155067,155067,38824,미성동458,1,서울특별시 관악구 난곡로40길 10,서울특별시 관악구 신림동 607-85,37.475267,126.917485,NaN,한전주,02-879-6782,서울특별시 관악구청,2022-09-22,서울관악경찰서,서울관악,G30000001329692,서울청,신림동
100039,100039,186832,2008-04-17,1,서울특별시 중구 다산로39길 20,서울특별시 중구 무학동 48,37.564535,127.014840,2010.0,한전주,02-3396-6152,중구청 도로시설과,2021-10-18,서울중부경찰서,서울중부,G30000001329261,서울청,무학동
20387,20387,82138,06-22-40,1,서울특별시 도봉구 시루봉로13길 54,서울특별시 도봉구 방학제2동 608-50,37.668416,127.030192,2015.0,전용주,02-2091-4083,도봉구청,2018-05-01,서울도봉경찰서,서울도봉,G30000001329939,서울청,방학동
177527,177527,69393,시흥2동 247,1,서울특별시 금천구 탑골로 55,서울특별시 금천구 시흥동 230-84 월편,37.452450,126.916178,NaN,한전주,02-2627-2405,서울특별시 금천구청,2022-02-26,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동
178453,178453,70322,시흥4동 658,1,서울특별시 금천구 버드나무4길 14-2 버드나무길,서울특별시 금천구 시흥동 1-70 유호그린빌라 마동,37.463184,126.911513,NaN,한전주,02-2627-2405,서울특별시 금천구청,2022-02-26,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동
175959,175959,67816,독산본동 3052,1,서울특별시 금천구 독산로,서울특별시 금천구 독산4동 944-2,37.467263,126.905383,NaN,전용주,02-2627-2405,서울특별시 금천구청,2022-02-26,서울금천경찰서,서울금천,G30000001332675,서울청,독산동


In [25]:
gu_pattern = r'(\S+구)'  # "관악구"를 추출하기 위한 정규표현식 패턴

from tqdm import tqdm

def extract_gu(address):
    match = re.search(gu_pattern, address)
    if match:
        return match.group(1)
    else:
        return None

tqdm.pandas()
light_sample["gu"] = light_sample["소재지도로명주소"].progress_apply(extract_gu)

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 5423.20it/s]


In [26]:
light_sample

,Unnamed: 0,번호,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong,gu
77699,77699,52914,자양4동223,1,서울특별시 광진구 뚝섬로22가길 58-5,서울특별시 광진구 자양동 57-65,37.534648,127.064156,NaN,한전주,02-450-7873,서울특별시 광진구청,2022-07-01,서울광진경찰서,서울광진,G30000001332648,서울청,자양동,광진구
190439,190439,149936,도림동-597,1,서울특별시 영등포구 도신로15다길 3,서울특별시 영등포구 도림동 232-12,37.507050,126.898234,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,도림동,영등포구
17758,17758,79506,2009-10-03,1,서울특별시 도봉구 창제1동 661-24,서울특별시 도봉구 창제1동 661-24,37.649755,127.035970,2008.0,한전주,02-2091-4083,도봉구청,2018-05-01,서울도봉경찰서,서울도봉,G30000001329939,서울청,창동,도봉구
137261,137261,103554,홍제1동,1,서울특별시 서대문구 통일로39길 80-1,서울특별시 서대문구 홍제1동312-94,37.586962,126.940749,NaN,건축물,02-330-1275,서울특별시 서대문구청,2020-01-01,서울서대문경찰서,서울서대문,G30000001329309,서울청,홍제동,서대문구
155067,155067,38824,미성동458,1,서울특별시 관악구 난곡로40길 10,서울특별시 관악구 신림동 607-85,37.475267,126.917485,NaN,한전주,02-879-6782,서울특별시 관악구청,2022-09-22,서울관악경찰서,서울관악,G30000001329692,서울청,신림동,관악구
100039,100039,186832,2008-04-17,1,서울특별시 중구 다산로39길 20,서울특별시 중구 무학동 48,37.564535,127.014840,2010.0,한전주,02-3396-6152,중구청 도로시설과,2021-10-18,서울중부경찰서,서울중부,G30000001329261,서울청,무학동,중구
20387,20387,82138,06-22-40,1,서울특별시 도봉구 시루봉로13길 54,서울특별시 도봉구 방학제2동 608-50,37.668416,127.030192,2015.0,전용주,02-2091-4083,도봉구청,2018-05-01,서울도봉경찰서,서울도봉,G30000001329939,서울청,방학동,도봉구
177527,177527,69393,시흥2동 247,1,서울특별시 금천구 탑골로 55,서울특별시 금천구 시흥동 230-84 월편,37.452450,126.916178,NaN,한전주,02-2627-2405,서울특별시 금천구청,2022-02-26,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동,금천구
178453,178453,70322,시흥4동 658,1,서울특별시 금천구 버드나무4길 14-2 버드나무길,서울특별시 금천구 시흥동 1-70 유호그린빌라 마동,37.463184,126.911513,NaN,한전주,02-2627-2405,서울특별시 금천구청,2022-02-26,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동,금천구
175959,175959,67816,독산본동 3052,1,서울특별시 금천구 독산로,서울특별시 금천구 독산4동 944-2,37.467263,126.905383,NaN,전용주,02-2627-2405,서울특별시 금천구청,2022-02-26,서울금천경찰서,서울금천,G30000001332675,서울청,독산동,금천구


### 숫자 이후 버림

In [37]:
cleaned_address = re.sub(r'\d.*', '', light_sample["소재지지번주소"].iloc[0])

In [38]:
cleaned_address

'서울특별시 광진구 자양동 '

In [39]:
def remove_digits_after(address):
    return re.sub(r'\d.*', '', address)

tqdm.pandas()
light_sample["정제된주소"] = light_sample["소재지지번주소"].progress_apply(remove_digits_after)

100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 16474.09it/s]


In [41]:
def extract_gu(address):
    match = re.search(gu_pattern, address)
    if match:
        return match.group(1)
    else:
        return None

tqdm.pandas()
light_sample["gu"] = light_sample["정제된주소"].progress_apply(extract_gu)

100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 18616.53it/s]


In [42]:
light_sample

,Unnamed: 0,번호,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong,gu,정제된주소
77699,77699,52914,자양4동223,1,서울특별시 광진구 뚝섬로22가길 58-5,서울특별시 광진구 자양동 57-65,37.534648,127.064156,NaN,한전주,02-450-7873,서울특별시 광진구청,2022-07-01,서울광진경찰서,서울광진,G30000001332648,서울청,자양동,광진구,서울특별시 광진구 자양동
190439,190439,149936,도림동-597,1,서울특별시 영등포구 도신로15다길 3,서울특별시 영등포구 도림동 232-12,37.507050,126.898234,NaN,한전주,02-2670-3838,서울특별시 영등포구,2022-06-27,서울영등포경찰서,서울영등포,G30000001329457,서울청,도림동,영등포구,서울특별시 영등포구 도림동
17758,17758,79506,2009-10-03,1,서울특별시 도봉구 창제1동 661-24,서울특별시 도봉구 창제1동 661-24,37.649755,127.035970,2008.0,한전주,02-2091-4083,도봉구청,2018-05-01,서울도봉경찰서,서울도봉,G30000001329939,서울청,창동,도봉구,서울특별시 도봉구 창제
137261,137261,103554,홍제1동,1,서울특별시 서대문구 통일로39길 80-1,서울특별시 서대문구 홍제1동312-94,37.586962,126.940749,NaN,건축물,02-330-1275,서울특별시 서대문구청,2020-01-01,서울서대문경찰서,서울서대문,G30000001329309,서울청,홍제동,서대문구,서울특별시 서대문구 홍제
155067,155067,38824,미성동458,1,서울특별시 관악구 난곡로40길 10,서울특별시 관악구 신림동 607-85,37.475267,126.917485,NaN,한전주,02-879-6782,서울특별시 관악구청,2022-09-22,서울관악경찰서,서울관악,G30000001329692,서울청,신림동,관악구,서울특별시 관악구 신림동
100039,100039,186832,2008-04-17,1,서울특별시 중구 다산로39길 20,서울특별시 중구 무학동 48,37.564535,127.014840,2010.0,한전주,02-3396-6152,중구청 도로시설과,2021-10-18,서울중부경찰서,서울중부,G30000001329261,서울청,무학동,중구,서울특별시 중구 무학동
20387,20387,82138,06-22-40,1,서울특별시 도봉구 시루봉로13길 54,서울특별시 도봉구 방학제2동 608-50,37.668416,127.030192,2015.0,전용주,02-2091-4083,도봉구청,2018-05-01,서울도봉경찰서,서울도봉,G30000001329939,서울청,방학동,도봉구,서울특별시 도봉구 방학제
177527,177527,69393,시흥2동 247,1,서울특별시 금천구 탑골로 55,서울특별시 금천구 시흥동 230-84 월편,37.452450,126.916178,NaN,한전주,02-2627-2405,서울특별시 금천구청,2022-02-26,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동,금천구,서울특별시 금천구 시흥동
178453,178453,70322,시흥4동 658,1,서울특별시 금천구 버드나무4길 14-2 버드나무길,서울특별시 금천구 시흥동 1-70 유호그린빌라 마동,37.463184,126.911513,NaN,한전주,02-2627-2405,서울특별시 금천구청,2022-02-26,서울금천경찰서,서울금천,G30000001332675,서울청,시흥동,금천구,서울특별시 금천구 시흥동
175959,175959,67816,독산본동 3052,1,서울특별시 금천구 독산로,서울특별시 금천구 독산4동 944-2,37.467263,126.905383,NaN,전용주,02-2627-2405,서울특별시 금천구청,2022-02-26,서울금천경찰서,서울금천,G30000001332675,서울청,독산동,금천구,서울특별시 금천구 독산


In [55]:
# 전체 데이터프레임을 처리할 범위 지정
# start_index = 0
end_index = 30000

# subset_df 생성
subset_df = light.iloc[:end_index]

In [56]:
subset_df.head(3)

,Unnamed: 0,번호,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치년도,설치형태,관리기관전화번호,관리기관명,데이터기준일자,경찰서명,경찰서부서명,경찰서부서코드,시도경찰청명,dong,gu
0,0,10212,강일동068,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동,37.576149,127.169137,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동,강동구
1,1,10213,강일동069,1,서울특별시 강동구 아리수로93가길 223,서울특별시 강동구 강일동,37.576176,127.168800,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동,강동구
2,2,10214,강일동070,1,서울특별시 강동구 아리수로93가길,서울특별시 강동구 강일동,37.576840,127.168722,NaN,한전주,02-3425-6373,서울특별시 강동구청,2021-08-25,서울강동경찰서,서울강동,G30000001329742,서울청,강일동,강동구


In [57]:
from tqdm import tqdm

def get_gu_name(row, sub_gu_error):
    try: 
        location = get_location(row["경도"], row["위도"])
        return location["documents"][0]["address"]["region_2depth_name"]
    except IndexError:
        sub_gu_error.append([row["경도"], row["위도"]])
        return None

tqdm.pandas()
sub_gu_error = []
subset_df["gu"] = subset_df.progress_apply(lambda row: get_gu_name(row, sub_gu_error), axis=1)

 55%|██████████████████████████████████████████▋                                  | 16615/30000 [37:49<30:28,  7.32it/s]


KeyboardInterrupt: 

In [ ]:
add_light = light[:]
add_light

In [ ]:
gu_list = subset_df["gu"].tolist()
gu_list[-20:]

In [ ]:
add_light.iloc[:end_index, add_light.columns.get_loc("gu")] = gu_list

In [ ]:
light.iloc[:end_index, light.columns.get_loc("gu")] = gu_list
light